# Stoic Citadel - Research Lab
## Strategy Development Template with VectorBT

**Philosophy:** *"In research, we seek truth. In trading, we execute truth."*

---

### Workflow:
1. **Load Data** - Import historical price data
2. **Calculate Indicators** - Technical analysis
3. **Generate Signals** - Buy/Sell logic
4. **Backtest** - Test strategy on historical data (VectorBT)
5. **Analyze** - Performance metrics, drawdown, Sharpe ratio
6. **Optimize** - Parameter tuning
7. **Export** - Convert to Freqtrade strategy

---

In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================

import pandas as pd
import numpy as np
import vectorbt as vbt
import ccxt
from pathlib import Path
import json
from datetime import datetime, timedelta

# Plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Technical Analysis
import talib as ta
import pandas_ta as pta

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ All libraries loaded successfully")
print(f"VectorBT version: {vbt.__version__}")

## 1. Load Data

We'll load data from Freqtrade's data directory or fetch it directly from exchange.

In [ ]:
# =============================================================================
# DATA LOADING
# =============================================================================

def load_freqtrade_data(pair: str, timeframe: str = '5m') -> pd.DataFrame:
    """
    Load data from Freqtrade's data directory.
    
    Args:
        pair: Trading pair (e.g., 'BTC/USDT')
        timeframe: Candle timeframe (e.g., '5m', '1h')
    
    Returns:
        DataFrame with OHLCV data
    """
    pair_filename = pair.replace('/', '_')
    data_path = Path(f'../user_data/data/binance/{pair_filename}-{timeframe}.json')
    
    if not data_path.exists():
        print(f"⚠️  Data not found: {data_path}")
        print("💡 Download data first using: docker-compose run --rm freqtrade download-data")
        return None
    
    with open(data_path, 'r') as f:
        data = json.load(f)
    
    df = pd.DataFrame(
        data,
        columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']
    )
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    
    print(f"✅ Loaded {len(df)} candles for {pair}")
    print(f"📅 Date range: {df.index[0]} to {df.index[-1]}")
    
    return df


def fetch_data_from_exchange(
    exchange_name: str = 'binance',
    symbol: str = 'BTC/USDT',
    timeframe: str = '5m',
    since: str = '2024-01-01',
    limit: int = 1000
) -> pd.DataFrame:
    """
    Fetch data directly from exchange using CCXT.
    
    Args:
        exchange_name: Exchange name (e.g., 'binance')
        symbol: Trading pair (e.g., 'BTC/USDT')
        timeframe: Candle timeframe (e.g., '5m', '1h')
        since: Start date (ISO format)
        limit: Number of candles to fetch
    
    Returns:
        DataFrame with OHLCV data
    """
    exchange_class = getattr(ccxt, exchange_name)
    exchange = exchange_class({'enableRateLimit': True})
    
    since_timestamp = int(datetime.fromisoformat(since).timestamp() * 1000)
    
    print(f"📡 Fetching {symbol} data from {exchange_name}...")
    
    ohlcv = exchange.fetch_ohlcv(
        symbol,
        timeframe=timeframe,
        since=since_timestamp,
        limit=limit
    )
    
    df = pd.DataFrame(
        ohlcv,
        columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']
    )
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    
    print(f"✅ Fetched {len(df)} candles")
    
    return df


# =============================================================================
# LOAD YOUR DATA
# =============================================================================

# Option 1: Load from Freqtrade data directory
# df = load_freqtrade_data('BTC/USDT', '5m')

# Option 2: Fetch from exchange
df = fetch_data_from_exchange(
    exchange_name='binance',
    symbol='BTC/USDT',
    timeframe='5m',
    since='2024-01-01',
    limit=1000
)

# Display sample
df.head()

## 2. Calculate Indicators

Add technical indicators to the dataframe.

In [ ]:
# =============================================================================
# INDICATORS
# =============================================================================

def add_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add technical indicators to dataframe.
    """
    df = df.copy()
    
    # Moving Averages
    df['ema_50'] = ta.EMA(df['close'], timeperiod=50)
    df['ema_200'] = ta.EMA(df['close'], timeperiod=200)
    
    # RSI
    df['rsi'] = ta.RSI(df['close'], timeperiod=14)
    
    # MACD
    macd, macdsignal, macdhist = ta.MACD(
        df['close'],
        fastperiod=12,
        slowperiod=26,
        signalperiod=9
    )
    df['macd'] = macd
    df['macdsignal'] = macdsignal
    df['macdhist'] = macdhist
    
    # Bollinger Bands
    upper, middle, lower = ta.BBANDS(
        df['close'],
        timeperiod=20,
        nbdevup=2,
        nbdevdn=2
    )
    df['bb_upper'] = upper
    df['bb_middle'] = middle
    df['bb_lower'] = lower
    
    # ATR (volatility)
    df['atr'] = ta.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    
    # ADX (trend strength)
    df['adx'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    
    print("✅ Indicators calculated")
    
    return df


# Add indicators
df = add_indicators(df)

# Drop NaN rows
df.dropna(inplace=True)

print(f"✅ Dataset ready: {len(df)} rows")
df.tail()

## 3. Generate Signals

Define entry and exit conditions.

In [ ]:
# =============================================================================
# SIGNAL GENERATION
# =============================================================================

# Entry signal: RSI oversold + Price above EMA200
entry_signal = (
    (df['rsi'] < 30) &
    (df['close'] > df['ema_200']) &
    (df['adx'] > 20)
)

# Exit signal: RSI overbought
exit_signal = (
    (df['rsi'] > 70)
)

print(f"✅ Entry signals: {entry_signal.sum()}")
print(f"✅ Exit signals: {exit_signal.sum()}")

## 4. Backtest with VectorBT

**VectorBT Magic:** Test years of data in seconds using vectorized operations.

In [ ]:
# =============================================================================
# VECTORBT BACKTEST
# =============================================================================

# Create portfolio
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    entries=entry_signal,
    exits=exit_signal,
    init_cash=10000,
    fees=0.001,  # 0.1% trading fee
    slippage=0.0005,  # 0.05% slippage
    freq='5min'
)

print("✅ Backtest completed!")
print("\n" + "="*50)
print("PERFORMANCE SUMMARY")
print("="*50)
print(portfolio.stats())

## 5. Analyze Results

Visualize performance and risk metrics.

In [ ]:
# =============================================================================
# VISUALIZATION
# =============================================================================

# Plot portfolio value
fig = portfolio.plot()
fig.show()

# Plot drawdown
fig_dd = portfolio.drawdowns.plot()
fig_dd.show()

# Plot trade PnL
fig_trades = portfolio.trades.plot()
fig_trades.show()

In [ ]:
# =============================================================================
# KEY METRICS
# =============================================================================

total_return = portfolio.total_return()
sharpe_ratio = portfolio.sharpe_ratio()
max_drawdown = portfolio.max_drawdown()
win_rate = portfolio.trades.win_rate()
total_trades = portfolio.trades.count()

print("\n" + "="*50)
print("KEY PERFORMANCE INDICATORS")
print("="*50)
print(f"💰 Total Return: {total_return:.2%}")
print(f"📊 Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"📉 Max Drawdown: {max_drawdown:.2%}")
print(f"🎯 Win Rate: {win_rate:.2%}")
print(f"🔢 Total Trades: {total_trades}")
print("="*50)

# The Stoic Verdict
if sharpe_ratio > 2 and max_drawdown > -0.15:
    print("\n✅ STOIC APPROVED: Strategy shows promise")
elif sharpe_ratio > 1:
    print("\n⚠️  NEEDS OPTIMIZATION: Strategy has potential but needs work")
else:
    print("\n❌ REJECTED: Strategy does not meet Stoic standards")

## 6. Parameter Optimization

Find optimal parameters using grid search.

In [ ]:
# =============================================================================
# PARAMETER OPTIMIZATION
# =============================================================================

# Define parameter ranges
rsi_entry_range = np.arange(20, 40, 5)
rsi_exit_range = np.arange(60, 80, 5)

results = []

print("🔍 Running optimization...")

for rsi_entry in rsi_entry_range:
    for rsi_exit in rsi_exit_range:
        # Generate signals
        entry = (
            (df['rsi'] < rsi_entry) &
            (df['close'] > df['ema_200']) &
            (df['adx'] > 20)
        )
        exit_ = (df['rsi'] > rsi_exit)
        
        # Backtest
        pf = vbt.Portfolio.from_signals(
            close=df['close'],
            entries=entry,
            exits=exit_,
            init_cash=10000,
            fees=0.001,
            freq='5min'
        )
        
        # Store results
        results.append({
            'rsi_entry': rsi_entry,
            'rsi_exit': rsi_exit,
            'total_return': pf.total_return(),
            'sharpe_ratio': pf.sharpe_ratio(),
            'max_drawdown': pf.max_drawdown(),
            'win_rate': pf.trades.win_rate(),
            'total_trades': pf.trades.count()
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Sort by Sharpe Ratio
results_df = results_df.sort_values('sharpe_ratio', ascending=False)

print("\n✅ Optimization complete!")
print("\nTop 5 parameter combinations:")
print(results_df.head(10))

## 7. Heatmap Visualization

Visualize how parameters affect performance.

In [ ]:
# =============================================================================
# HEATMAP
# =============================================================================

# Pivot for heatmap
heatmap_data = results_df.pivot(
    index='rsi_entry',
    columns='rsi_exit',
    values='sharpe_ratio'
)

# Plot
plt.figure(figsize=(12, 8))
sns.heatmap(
    heatmap_data,
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0
)
plt.title('Sharpe Ratio Heatmap: RSI Entry vs Exit Thresholds', fontsize=14)
plt.xlabel('RSI Exit Threshold', fontsize=12)
plt.ylabel('RSI Entry Threshold', fontsize=12)
plt.show()

## 8. Export Strategy

Once you've found a winning strategy, export it to Freqtrade format.

In [ ]:
# =============================================================================
# EXPORT TO FREQTRADE
# =============================================================================

print("""
To export this strategy to Freqtrade:

1. Open: ../user_data/strategies/StoicEnsembleStrategy.py
2. Update the RSI parameters in populate_entry_trend() and populate_exit_trend()
3. Test in dry-run mode:
   docker-compose run --rm freqtrade backtesting --strategy StoicEnsembleStrategy
4. If successful, enable live trading in config_production.json

Remember: Past performance does not guarantee future results.
"The wise trader tests thoroughly before risking capital."
""")

---

## Next Steps

1. **Try different indicators**: MACD, Bollinger Bands, etc.
2. **Add ML validation**: Use XGBoost to filter signals
3. **Test on multiple pairs**: Diversification
4. **Walk-forward optimization**: Avoid overfitting
5. **Paper trade**: Test in real-time with fake money

**The Stoic Way:** Discipline, patience, risk management.

---